# Optimizing!

Before we dive into modeling the ESUPS optimization problem, here is quick overview of some sub-notebooks provided:
* 3.A Introduction to Modeling Optimization Problems
* 3.B More on Expected Value

## Intro to the ESUPS Optimization Model

We'll start with a simplified version of the model, and we'll build from there.

Consider the scenario where we're expecting to have a specific disaster and want to transport the supplies from warehouses with predefined stock as quickly as possible to the disaster. Perhaps it's hurricane season or a volcano has been showing increasing activity. As we have seen in past problems, we have to set up the variables, constraints, and an objective function.

First, we list the input data we have:
- $I$ is the set of warehouses
- $\tau_i$ is the driving time to ship a single item from warehouse $i \in I$ to the disaster
- $b_i$ is the starting inventory (buckets) at each warehouse $i \in I$
- $d$ is the demand we need to satisfy, i.e., the number of items we need to ship to the disaster

Then, let's define our decision variables, i.e. the things we can vary.
- $y_i$ is the amount of supplies to send from warehouse $i \in I$ to our disaster

Now that we have described what we can change with our variables, we can figure out how to represent the objective function!
$$
\min \sum_{i \in I} \tau_{i} \cdot y_i
$$
and the constraints are:
$$
\begin{aligned}
\text{s.t.}  & \sum_{i \in I} & y_{i} & = d & & \quad \text{(total supplies sent must meet demand)}\\
& & y_i       & \leq b_i & \forall i \in I & \quad \text{(you can't send more than a warehouse has)}\\
 &\text{} & y_{i} & \geq 0 &\forall i \in I & \quad \text{(you can't send negative supplies)}
\end{aligned}
$$

### Make the Model
So now let's code and solve this model. 

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

simple_Allocation = pd.read_csv("data/simple_Allocation.csv")

# Prep data
BucketsNeeded = 13561
n = len(simple_Allocation)
t = simple_Allocation.drivingTime_hrs
b = simple_Allocation.Buckets

# Create model
model = gp.Model("simple_Allocation")

# Add decision variables
y = model.addVars(n, name="Warehouse_Allocation")

# Add constraint to meet demand
model.addConstr(
    gp.quicksum(y[i] for i in range(n)) == BucketsNeeded, name="Meet_Demand"
)

# Add in warehouse_constraints
for i in range(n):
    model.addConstr(y[i] <= b[i], name=f"warehouse_endowment_{i}")

# Note we don't have a constraint for y >= 0 since it's assumed in the variable definition
# Add objective
objective = gp.quicksum(t[i] * y[i] for i in range(n))
model.setObjective(objective, GRB.MINIMIZE)

# Fire up the solver!
model.optimize()

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x37d66425
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+04]
Presolve removed 16 rows and 2 columns
Presolve time: 0.01s
Presolved: 1 rows, 14 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.691875e+03   0.000000e+00      0s
       1    9.8293000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.829300000e+04


Now Let's Anaylize the results!

In [2]:
simple_Allocation_sol = simple_Allocation.copy()
simple_Allocation_sol["y"] = model.getAttr("X", y)
simple_Allocation_sol.sort_values("drivingTime_hrs")

,depotCity,drivingTime_hrs,Buckets,y
1,Ambatondrazaka,0.00,26,26.0
4,Antananarivo Renivohitra,6.00,9046,9046.0
11,Miarinarivo,7.00,3,3.0
14,Toamasina I,8.00,1580,1580.0
5,Antsohihy,10.00,610,610.0
2,Ambositra,11.00,41,0.0
7,Fenerive Est,11.00,6689,2296.0
0,Ambanja,14.00,375,0.0
9,Maintirano,14.25,2460,0.0
8,Mahajanga I,16.00,150,0.0


Now you may have noticed that this feels like overkill. If we want to position supplies to respond to a known disaster, you might think that we should just put them as close as possible. It's an intuitive solution that can be solved with a simple greedy algorithm. But of course, life is never that simple. 

Now that we've got the initial problem outlined, let's start making it more realistic with two additions:
1.	Instead of preparing for only one disaster, let's prepare for all the disasters that might occur.
2.	Instead of being an omniscient observer, let's say we aren't sure where the next disaster will be

Now, these assumptions might be intimidating at first. How do you rigorously model information we don't know? Isn't this the realm of predictive models?

This reaction is totally normal and indeed we often use predictive modeling when our information is incomplete, however, remember that this problem isn't easily suited to ML techniques due to the low availability of data, small feature space, and large solution space. So how do we approach this with optimization? Well now that we know how supplies will be dispensed for a given warehouse allocation, we can leverage the small set of historical disasters we do have to ask which allocations would have saved/minimized the time taken to respond to all of the disasters. Then our optimization techniques will give us a small number that could feasibly satisfy this (where our constraints meet, see the introduction for more on this) and we can just iterate over them.

So how do we go about it in practice?

## Including All Disasters

Before we begin, let's explicitly define our new problem with the additional requirements outlined in the previous section so we're all on the same page. Our first step is to add in the fact that there are more disasters than just one. We can do that by including a variable index to denote which disaster we're talking about. 

Let $K$ be the set of disasters, and $k$ be the index for the disaster scenario at hand, i.e., a storm, earthquake, or epidemic.

In scenario $k$, the time for a warehouse $i$ to send $y_i$ items is 
$$\tau_{ik} \cdot y^k_i$$

Repeating for all warehouses gives us
$$  \sum_i \tau_{ik} \cdot y^k_i $$

Repeating for all disasters gives us
$$ \sum_k \sum_i \tau_{ik} \cdot y^k_i $$

The question that ESUPS, and subsequently this model, set out to solve is: Where should disaster relief supplies be stored in a country to ensure it is as fast and cheap as possible to get them to people in need in the event of a disaster? 

By comparing the overall travel time for every single disaster that we have data on, we can make determinations on whether storing different quantities of materials in different warehouses would be better or worse given our historical data.

Remember that the model we're looking at specifically optimizes for time, to solve for cost we would replace our time value $^{\tau_{ik}}$ with a cost value $^{c_{ik}}$.

Before we get to solving, let's add in one more factor. Different disasters occur at different rates. A landlocked nation may be less likely to experience a disaster caused by a tropical storm than an earthquake, so shouldn't we weigh the response time to earthquakes more than that of a storm?

### Accounting for Randomness

One of the most difficult tasks that we can encounter in data science problems is randomness or as it's often called "stochastic" elements. This can be seen in all kinds of ways, but in our case study, we're going to look at how we account for not knowing which disaster will hit next or even several years in the future. The way we do this is to create a stochastic model, which may initially seem intimidating, but in discrete events like this, it's super easy.

If you're familiar with expected value this will quickly make sense, but no need to have any prior experience! See the sub-notebook 3.B for a quick intro.
The idea is that we weigh the outcome of the event (i.e. total travel time in this case) by the probability it occurs. So, if an earthquake is 3 times as likely as a flood, we would rewrite the equation as:

$$0.75 \cdot\text{(travel time earthquake)} + 0.25 \cdot\text{(travel time flood)} $$

#### The Model

Now that we've had some background on expected value, let's return to our model! Remember, our goal is to account for the inherent randomness associated with different disasters occuring.

Let $P^k$ be the probability of disaster $k$ and $t^k$ be the total travel time involved in disaster $k$ from the previous sections. Using our definition of expected value, this gives us

$$ \sum_k P^k \cdot t^k$$

Now you might notice that we have already written an equation for the total travel time for disaster $k$. Substituting this in, we get

$$\sum_k P^k \sum_i \tau_{ik} \cdot y^k_i$$

So, you can see, in our case turning this problem into a stochastic optimization problem that can account for probability only involves one more value per disaster. Our final task right now is to minimize the time required to get supplies to the disasters given how likely each disaster is to occur:

$$
\min_y \sum_k P^k \sum_i \tau_{ik} \cdot y^k_i
$$

And we can see the constraints are almost unchanged:
$$
\begin{aligned}
\text{s.t.} \quad \sum_{i} y^k_{i} & = d^k & \forall k & \quad \text{(total supplies sent must meet demand)}\\
y^k_i & \leq b_i & \forall i \in I,~ \forall k \in K & \quad \text{(you can't send more than a warehouse has)}\\
y^k_{i} & \geq 0 & \forall i \in I,~ \forall k \in K & \quad \text{(you can't send negative supplies)}\\
\end{aligned}
$$

Note that we made an important assumption when defining the warehouse capacity constraints: We assumed that not all disasters happen at the same time, and so we have the full warehouse capacity available for each disaster.

All the probabilities should sum up to one, i.e., $\sum_k P^k = 1$. How do we calculate the probabilities though? Well, we have good long-term data on what disasters have affected which countries. For now, we can go through that data and calculate the probability of disaster $k$ by counting how many times it has occurred and dividing by the number of total disasters.

You may have noticed that every single disaster will have the same probability, or in other words, this will produce a uniform distribution. As a result, $P^k$ can effectively be removed from our objective function, so our new objective will give the exact same results as our old one. We'll talk about ways to potentially address this later on in the notebooks as an extension, but for the moment, the purpose of adding this is to show how easily the equation can be modified to be stochastic and to get users used to the idea.
So now that we have the new objective function, we can let the model go ahead and let Gurobi solve it for us!

However, before we need to read in the data and prepare it!

In [3]:
warehouses = pd.read_csv("data/warehouses.csv")
warehouses

,Type,Lat,Long,Buckets
0,Warehouses,-17.823700,48.426300,26
1,Warehouses,-20.516700,47.250000,41
2,Warehouses,-18.908500,47.537500,9046
3,Warehouses,-17.384300,49.409800,2762
4,Warehouses,-16.916700,49.900000,1682
5,Warehouses,-15.430900,49.758300,1870
6,Warehouses,-16.170200,49.774100,0
7,Warehouses,-16.926100,49.587100,375
8,Warehouses,-25.176133,46.089378,2322
9,Warehouses,-25.031600,46.990000,0


In [4]:
distanceMatrix_scenarios = pd.read_csv("data/distanceMatrix_scenarios.csv")
distanceMatrix_scenarios

,scenario,warehouse,drivingTime_hrs
0,0,0,21.00
1,0,1,30.00
2,0,2,25.00
3,0,3,32.00
4,0,4,35.75
...,...,...,...
457,21,16,26.00
458,21,17,19.00
459,21,18,48.00
460,21,19,33.00


In [5]:
disasters = pd.read_csv("data/disasters.csv")
# one bucket is needed for 2.5 people on average
total_buckets = warehouses["Buckets"].sum()
disasters["demand"] = (
    (disasters["People Impacted"] / 2.5).round().clip(upper=total_buckets).astype(int)
)
demand = disasters["demand"].values
probs = [1 / len(demand)] * len(demand)

disasters

,Type,Lat,Long,People Impacted,demand
0,Storm,-12.266700,49.283300,118000,40811
1,Storm,-14.266700,50.166700,100215,40086
2,Epidemic,-14.876200,47.983500,21976,8790
3,Epidemic,-15.716700,46.316700,15172,6069
4,Flood,-16.950400,46.828100,20000,8000
5,Flood,-17.384300,49.409800,28223,11289
6,Storm,-17.823700,48.426300,84309,33724
7,Storm,-18.064600,44.029500,526200,40811
8,Storm,-18.149900,49.402300,600000,40811
9,Epidemic,-18.769800,46.050000,3055,1222


In [6]:
m = len(demand)  # number of disasters/scenarios
t = distanceMatrix_scenarios.set_index(["scenario", "warehouse"]).squeeze().to_dict()
b = warehouses["Buckets"][warehouses["Buckets"] > 0].reset_index(drop=True)
n = len(b)  # number of warehouses

model_2 = gp.Model("multiple_disasters")

# Amount to take per warehouse
y = model_2.addVars(t.keys(), name="Warehouse_Allocation")

# Add constraints to meet demand for each disaster scenario k
for k in range(m):

    # Demand constraints
    model_2.addConstr(y.sum(k, "*") == demand[k], name=f"Meet_Demand_K:{k}")

    # Warehouse constraints
    for i in range(n):
        model_2.addConstr(y[k, i] <= b[i], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time over all scenarios
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[k, i] * y[k, i] for i in range(n)) for k in range(m)
)

# Optimize model
model_2.setObjective(objective, GRB.MINIMIZE)
model_2.optimize()

# Store results in dataframe a
a = pd.DataFrame(
    {
        "Warehouse allocation": model_2.getAttr("VarName", y),
        "Buckets": model_2.getAttr("X", y),
    }
)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 484 rows, 462 columns and 924 nonzeros
Model fingerprint: 0xa84de061
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 4e+04]
Presolve removed 478 rows and 372 columns
Presolve time: 0.00s
Presolved: 6 rows, 90 columns, 90 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1096854e+05   1.040175e+04   0.000000e+00      0s
       6    4.4697821e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.469782127e+05


In [7]:
a[:15]

Warehouse allocation  Buckets
0 0    Warehouse_Allocation[0,0]     26.0
  1    Warehouse_Allocation[0,1]     41.0
  2    Warehouse_Allocation[0,2]   9046.0
  3    Warehouse_Allocation[0,3]   2762.0
  4    Warehouse_Allocation[0,4]   1682.0
  5    Warehouse_Allocation[0,5]   1870.0
  6    Warehouse_Allocation[0,6]    375.0
  7    Warehouse_Allocation[0,7]   2322.0
  8    Warehouse_Allocation[0,8]    610.0
  9    Warehouse_Allocation[0,9]   1027.0
  10  Warehouse_Allocation[0,10]   1870.0
  11  Warehouse_Allocation[0,11]   3331.0
  12  Warehouse_Allocation[0,12]   1580.0
  13  Warehouse_Allocation[0,13]    150.0
  14  Warehouse_Allocation[0,14]    375.0

### 3.4 Data Science Extension

We've seen how the objective function uses the uniform distribution to solve the problem. But what if knowing that climate change is increasingly energizing large storms, we decide the past hurricane impacts aren't representative of what's to come? In this section we want to prompt you to come up with predictive elements to improve our models. Feel free to use some of the ideas below or go in an entirely new direction!

In this section, we encourage you to think creatively about enhancing predictive models for climate-related disasters. Consider how to incorporate novel data sources, feature engineering techniques, and model architectures to improve predictions. Below are some suggested approaches, but feel free to explore entirely new directions!

#### Case Study Focus: Coastal Eastern African Nations

Using the disaster impact data for coastal Eastern African nations, can you develop a model to predict how these impacts might escalate for Madagascar in the coming years? Consider not only the historical data but also factors such as changes in sea surface temperatures, shifting storm tracks, population growth along vulnerable coastlines, and evolving infrastructure resilience. Further, can you integrate this predictive model into an optimization framework to better allocate resources for disaster preparedness and response?

Potential Approaches to Explore:
- Comparing Time Series Models: Traditional statistical time series models like ARIMAX (AutoRegressive Integrated Moving Average with Explanatory Variables) are commonly used to predict future values based on past data. How do these models compare with more advanced Recurrent Neural Network (RNN)-based approaches like Long Short-Term Memory (LSTM) networks or Gated Recurrent Units (GRUs) in capturing long-term dependencies, especially under non-stationary conditions induced by climate change?

- Incorporating Geospatial Data: Geospatial features, such as latitude, longitude, elevation, and proximity to bodies of water, can play a crucial role in predicting the impact of tropical storms. Can we encode geospatial information using techniques like convolutional neural networks (CNNs) for spatial feature extraction, or leverage more specialized models such as Geographical Weighted Regression (GWR) or Graph Neural Networks (GNNs) to account for spatial dependencies?

- Incorporating Climate Change Projections: Beyond just historical data, consider how future climate projections can be integrated into the model. Can we use downscaled climate model outputs or ensemble approaches to account for different climate scenarios? How would these scenarios affect the frequency and intensity of tropical storms affecting coastal Eastern African nations?

- Feature Engineering with Climate Indicators: Introduce climate change indicators as predictive features. For example, how do trends in sea surface temperatures (SSTs), El Niño-Southern Oscillation (ENSO) phases, or the Atlantic Multi-decadal Oscillation (AMO) correlate with storm intensification? Would incorporating these indicators as additional time series variables enhance predictive accuracy?
	
- Hybrid and Ensemble Models: Can we leverage hybrid models that combine both statistical and deep learning approaches, or use ensemble methods that aggregate predictions from multiple models? For example, combining ARIMAX for short-term predictions with LSTM for capturing long-term trends may provide a more comprehensive forecasting tool.

- Optimization Integration: Once a reliable predictive model is established, how can it be integrated into an optimization framework for resource allocation? For example, can we build an optimization model that minimizes both the cost of disaster preparedness and the potential loss from future storm impacts?

- Model Explainability and Decision-Making: How can we ensure that the model is interpretable for decision-makers? Consider the use of techniques like SHAP (SHapley Additive exPlanations) values or LIME (Local Interpretable Model-agnostic Explanations) to explain which factors contribute most to the model’s predictions, helping policymakers make informed decisions.

By combining predictive analytics with optimization, we can not only forecast future disaster impacts but also develop actionable strategies for minimizing those impacts. The goal is to make our models both more accurate and more useful in real-world applications, driving better outcomes for communities at risk.


## Putting it All Together: Optimizing Allocation and Transportation

Great, now that we can solve for the time it takes to address every single disaster, we can finally answer the original question posed: How do we best allocate supplies to all the warehouses? The key here is to extend the optimization model. Remember how in our constraints we included that you can't send more than the warehouse has? 
$$y^k_i \leq b_i,~  \forall i \in I \quad \text{(you can't send more than a warehouse has)}$$

So far, we've just been using the actual allocation we have at each warehouse. But what if those changes? Suddenly we might have an entirely new solution. So, we say that the model's solution (i.e., the allocations $y^k_i \in Y$) is some function $Y=f(X)$ based on our warehouse starting inventories $x_i \in X$.

While this may initially look intimidating, it is one of the easiest changes to make to our current model. All we are doing is creating a decision variable $x_i \ge 0$ for each warehouse $i \in I$ instead of using constant values and limit the total amount of supplies over all warehouses by $\chi$.
Let's substitute back in our model from the last section with the new constraints to see this firsthand.

$$
\min_{x,y} \sum_k P^k \sum_i \tau_{ij}\cdot y^k_i
$$

Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 
$$
\begin{aligned}
\text{s.t.}  \sum_{i} y^k_{i} & = d^k & \forall k \in K & \quad\text{(total supplies sent must meet demand)}\\
y^k_i & \leq x_i & \forall i \in I, \forall k \in K & \quad \text{(you can't send more than a warehouse has)}\\
y^k_{i} & \geq 0 & \forall i \in I, \forall k \in K & \quad \text{(you can't send negative supplies)}\\
\hline \\
\sum_{i} x_i & = \chi & & \quad \text{(we allocate all supplies and no more)}\\
x_{i} & \geq 0 & \forall i \in I & \quad \text{(you can't allocate negative supplies)}
\end{aligned}
$$
So how do we implement this in Gurobi?

In [8]:
m = len(demand)  # number of disasters/scenarios
t = distanceMatrix_scenarios.set_index(["scenario", "warehouse"]).squeeze().to_dict()
b = warehouses["Buckets"][warehouses["Buckets"] > 0].reset_index(drop=True)
n = len(b)  # number of warehouses

model_full = gp.Model("full_allocation")

# Amount to take per warehouse
y = model_full.addVars(m, n, name="Warehouse_Allocation")

# Warehouse inventories
x = model_full.addVars(n, name="Warehouse_Inventory")

# Total national endowment constraint
model_full.addConstr(x.sum() == b.sum(), name="Total_National_Endowment")

# Add constraints to meet demand for each disaster scenario (k)
for k in range(m):

    model_full.addConstr(y.sum(k, "*") == demand[k], name=f"Meet_Demand_K:{k}")

    # Cannot allocate more than amount assigned to warehouse
    # x is now a decision variable
    for i in range(n):
        model_full.addConstr(y[k, i] <= x[i], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using t as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[k, i] * y[k, i] for i in range(n)) for k in range(m)
)

# Optimize model
model_full.setObjective(objective, GRB.MINIMIZE)
model_full.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 485 rows, 483 columns and 1407 nonzeros
Model fingerprint: 0x8817cae8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 4e+04]
Presolve time: 0.00s
Presolved: 485 rows, 483 columns, 1407 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4281409e+04   5.415380e+05   0.000000e+00      0s
     272    3.4027302e+05   0.000000e+00   0.000000e+00      0s

Solved in 272 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.402730155e+05


In [9]:
warehouse_inventories = model_full.getAttr("X", x)
warehouse_inventories

{0: 0.0,
 1: 18673.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 725.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 11289.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 10124.0}

In [10]:
y_values_full = pd.DataFrame(
    {
        "Warehouse allocation": model_full.getAttr("VarName", y),
        "Buckets": model_full.getAttr("X", y),
    }
)
y_values_full

Warehouse allocation  Buckets
0  0     Warehouse_Allocation[0,0]      0.0
   1     Warehouse_Allocation[0,1]  18673.0
   2     Warehouse_Allocation[0,2]      0.0
   3     Warehouse_Allocation[0,3]      0.0
   4     Warehouse_Allocation[0,4]      0.0
...                            ...      ...
21 16  Warehouse_Allocation[21,16]      0.0
   17  Warehouse_Allocation[21,17]      0.0
   18  Warehouse_Allocation[21,18]      0.0
   19  Warehouse_Allocation[21,19]      0.0
   20  Warehouse_Allocation[21,20]  10124.0

[462 rows x 2 columns]

## Extending the Problem Further

Now that we have our overall framework, we can extend it fairly easily to better model real-life scenarios. Let's look at two final pieces of the puzzle that ESUPS considers in their model:

**Cost**

Along with how long it takes to get items to a disaster relief site, it's also important to consider the cost to accomplish it. It might be a few hours faster to charter a jet to deliver blankets in the aftermath of a disaster, however, if it is 100x more expensive than by truck, that may constrain the organization from buying more blankets, chartering more trucks, or making it difficult to resupply for future disasters. So just as we solve for ways to minimize time, it can be important for firms with limited resources to make sure their money is being used to do the most good it can.

So how do we do this? It's fairly simple. Our time matrix, which we've been using to show how close or far buildings are from the disaster relief site, is just a set of predefined weights/discounts. So, if we change the numbers to reflect the cost of transit, then suddenly we're solving a cost-minimization problem! In fact, the substitution is so one-to-one, that besides switching $\tau_{ij}$ for $c_{ij}$, we don't have to change the equation.

**Travel Mode**

The second additional facet considered in our real-life model that we haven't encountered yet is transportation mode. We alluded to it a little in the cost section, but often there is the option to fly or ship goods into a region, which can be especially useful when far away or the roads are clogged or otherwise unusable (such is often the case after a disaster).

So how do we implement this? Well let's take a look back at $y_i^k$, our variable which says how many goods to send from warehouse $i$ to disaster $k$. All we want to do is reflect and updated description: how many goods to send from warehouse $i$ to disaster $k$ via mode $r$. This can easily be represented as $y_{ir}^k$, let's explain what's happened. Before is $y$ was an array of length $K$ with each index holding sub array of length $I$ (which we could also write as size $K \times I$), now each index in our subarrays also have an array of length $3$ to represent how much is sent via truck, plane, or boat. So our final array is of dimensions $K \times I \times R $. This may seem intimidating at first, but remember, adding a dimension just means adding one more nested for loop!

Let's look at how we would implement this. Remember, from a math point of view, all we've done is say $y_i^k$ can be broken down into $3$ modes instead of 1. So, it's rewritten as


$$
\min_{X,Y} \sum_k P^k \sum_i \sum_r \tau_{irj}\cdot y^k_{ir}
$$


Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 
$$
\begin{aligned}


\text{s.t.}  & \sum_{i}\sum_{r} & y^k_{ir}&=d^k & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& \sum_{r}  & y^k_{ir}       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y^k_{ir} &\geq 0 &\forall r\in R, i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}\\

& \sum_k & P^k &=1 && \hspace{.2cm} \text{(All probabilities must sum to 1)}\\


  & \sum_{i} & x_i&=\chi & & \hspace{.2cm} \text{(we allocate all supplies and no more)}\\

 && x_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't allocate negative supplies)}\\


\end{aligned}
$$

We've added a few more sums here, but remember, in math, a sum is just a for loop. $\sum_r$ is the equivalent to `for r in R:`. So how would we implement it in the format we've been using so far? This is going to be left as an open-ended exercise to the reader! If you get stuck, you can reference the production solver we'll be exploring below, which includes the mode of travel but is set up in a different approach than we've been using so far!

### Open-Ended Implementation

In [ ]:
# Write your code here

## Interpreting the Solution 

So let's look at how much slower our real life allocations are in comparison to the optimal.

### Enhancing System Performance with the Balance Metric

In humanitarian logistics, the efficiency of inventory allocation directly impacts the ability to respond swiftly and cost-effectively to disasters. The **Balance Metric** $(D)$ is a critical tool developed to evaluate the alignment of current inventory distribution with an optimal allocation. This metric is particularly valuable in contexts where multiple organizations independently manage inventory across various depots, without a centralized coordination mechanism.

#### Definition and Calculation of the Balance Metric

The balance metric $(D)$ is defined as the ratio between the actual objective value (either cost or time) of the current inventory allocation $V(X)$ and the optimal objective value $V(A')$ given the same overall capacity:

$$ D = \frac{V(A)}{V(A')}  $$

Here:
-  $V(A)$: Represents the current system-wide cost or time to meet demand based on the existing inventory allocation \(X\).
-  $V(A')$: Represents the minimized cost or time if the inventory were optimally distributed across all depots.

#### Interpretation of the Balance Metric

1. **Optimal Inventory Allocation**:
   The optimal value of $D$ is 1. This occurs when the current allocation perfectly aligns with the optimal allocation, meaning no further reallocation could reduce costs or response times.

2. **Identifying Imbalances**:
   When $D > 1$, the system is considered "out-of-balance." A value of 1.2, for example, implies that the current allocation incurs 20% higher costs or longer response times compared to an optimal arrangement. This indicates a potential for improvement by reallocating resources more effectively.

3. **Guiding System Improvements**:
   The balance metric is not only an indicator of inefficiency but also a guide for decision-making. By identifying locations or items with the highest imbalance, decision-makers can prioritize inventory reallocations that would yield the most significant improvements in terms of cost savings or faster response times.

#### Practical Applications in Humanitarian Logistics

The balance metric offers several practical applications for optimizing humanitarian response efforts:

- **Strategic Reallocation of Resources**:
  Organizations can use the balance metric to identify under-stocked or over-stocked depots and adjust inventory levels accordingly. This strategic reallocation can significantly enhance response times or reduce costs, especially in multi-organizational contexts where coordination is limited.

- **Sensitivity to Network Changes**:
  The balance metric is responsive to changes in the logistics network. For example, if a new depot is added in a high-risk area and remains under-stocked, the balance metric will reflect this imbalance, prompting an assessment of whether inventory should be redistributed to better leverage the new depot.

- **Decision-Making in Real-Time Operations**:
  By continuously monitoring the balance metric as part of a real-time dashboard, operational managers can be alerted to changes that may impact overall system performance. This enables them to make data-driven decisions quickly, improving the overall resilience and responsiveness of the humanitarian supply chain.

#### Limitations and Considerations

While the balance metric provides valuable insights into inventory allocation efficiency, it is important to consider its limitations:

- **Impact of Extreme Events**:
  The balance metric can be influenced by extreme scenarios, such as very large-scale disasters that significantly impact the calculated demand. As a result, it should be interpreted alongside other metrics, such as the fraction of demand served ($g$) or the weighted fraction of disasters completely served ($d$), to provide a more comprehensive picture of system performance.

- **Dependence on Data Quality and Model Assumptions**:
  The accuracy of the balance metric depends on the quality of input data and the assumptions made in the model. Ensuring robust and accurate data collection processes and regularly updating model parameters to reflect real-world conditions are essential for maintaining the reliability of the metric.

#### Conclusion

The balance metric $D$ offers a powerful tool for evaluating and improving the efficiency of inventory allocation in humanitarian logistics. By identifying imbalances and guiding strategic reallocation decisions, this metric can help organizations optimize their response efforts, ensuring that resources are used most effectively to meet the needs of affected populations during disasters.

<a id='End'></a>